In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_augmentation import random_transform


from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, MaxPooling2D, Reshape, Flatten, Input, merge
from keras import backend as K

Using TensorFlow backend.


In [2]:
resize_shape = (64, 64)

In [3]:
data = pd.read_csv("data/train.csv")

In [4]:
#For now, we remove new_whale
data = data[data['Id'] != 'new_whale']

In [5]:
len(data)

9040

In [6]:
from PIL import Image
image = Image.open('data/train/0a5c0f48.jpg')

In [7]:
image = image.resize(resize_shape)

In [8]:
# plt.imshow(image)
# plt.show()

In [9]:
np.array(image).shape

(64, 64, 3)

It's probably good to resize these images to (256, 256)

Its not sensible to read all the images into memory. So, we will read them in in batches of arbitrary size BATCH_SIZE (8?), and fit to them. Then do the same when predicting. Maybe use model.fit_generator? Generator can load files, preprocess and yield them.

In [10]:
gray = np.mean(image, -1)

In [11]:
gray.shape

(64, 64)

In [12]:
# plt.imshow(gray,cmap='gray')
# plt.show()

We need a map from a list of filenames and labels to an X, y.

In [13]:
def preprocess(image, size=resize_shape):
    image = image.resize(size)  
    image = np.array(image)
    if len(image.shape) == 3:
        image = np.mean(image, -1) 
    assert image.shape == resize_shape
    return image

def file_to_image(filename):
    raw_image = Image.open('data/train/' + filename)
    image = preprocess(raw_image)
    image = random_transform(image)
#     del raw_image
    return image

def file_list_to_image_array(file_list):
    return np.stack([file_to_image(f) for f in file_list])

In [14]:
files = data['Image']
y = data['Id']

#Use this to test simple classification
# y = y.map(lambda x: int(x=='new_whale'))

In [15]:
from sklearn.model_selection import train_test_split
files_train, files_test, y_train, y_test = train_test_split(
    files, y, test_size=0.1, random_state=42)

In [16]:
def batch_generator(iterable, batch_size=1):
    while True:
        l = len(iterable)
        for ndx in range(0, l, batch_size):
            yield iterable[ndx:min(ndx + batch_size, l)]

Now need to create the pairs of images inputs.

In [17]:
def get_same_whale_image(x):
    filtered = data[(data['Id'] == x['Id'])]
    return filtered.sample(1).values[0] 

def get_different_whale_image(x):
    filtered = data[(data['Id'] != x['Id'])]
    return filtered.sample(1).values[0]

In [18]:
BINARY_DATASET_SIZE = 10000

SAME_WHALE_DATASET_SIZE = BINARY_DATASET_SIZE // 2
DIFFERENT_WHALE_DATASET_SIZE = BINARY_DATASET_SIZE // 2

In [19]:
same_whale_dataset = data.sample(SAME_WHALE_DATASET_SIZE, replace=True)
same_whale_dataset[['Image2', 'Id2']] = same_whale_dataset.apply(get_same_whale_image, axis=1)
same_whale_dataset.columns = ['Image1', 'Id1','Image2', 'Id2']
same_whale_dataset = same_whale_dataset[same_whale_dataset['Image1'] != same_whale_dataset['Image2']]
same_whale_dataset['target'] = 1

In [20]:
different_whale_dataset = data.sample(DIFFERENT_WHALE_DATASET_SIZE, replace=True)
different_whale_dataset[['Image2', 'Id2']] = different_whale_dataset.apply(get_different_whale_image, axis=1)
different_whale_dataset.columns = ['Image1', 'Id1','Image2', 'Id2']
different_whale_dataset['target'] = 0

In [21]:
binary_dataset = pd.concat([same_whale_dataset, different_whale_dataset]).sample(frac=1)

In [22]:
len(binary_dataset)

7659

In [23]:
def binary_data_generator(batch_size):
    file_1_generator = batch_generator(binary_dataset['Image1'], batch_size)
    file_2_generator = batch_generator(binary_dataset['Image2'], batch_size)
    target_generator = batch_generator(binary_dataset['target'], batch_size)
    
    while True:
        file_1_batch = next(file_1_generator)
        file_2_batch = next(file_2_generator)
        
        image_1_batch = file_list_to_image_array(file_1_batch)
        image_2_batch = file_list_to_image_array(file_2_batch)
        
        pairs = [image_1_batch, image_2_batch]
        targets = next(target_generator)
        
        yield pairs, targets

In [24]:
BATCH_SIZE = 32
siamese_data_generator = binary_data_generator(BATCH_SIZE)

In [25]:
# next(siamese_data_generator)

This seems to be the correct generator.

# Create the network

In [ ]:
input_shape = resize_shape
left_input = Input(input_shape)
right_input = Input(input_shape)
#build convnet to use in each siamese 'leg'

convnet = Sequential()
convnet.add(Reshape((resize_shape[0],resize_shape[1], 1,), input_shape=(resize_shape[0],resize_shape[1],),name='Reshape'))
convnet.add(Conv2D(filters=8, input_shape=(resize_shape[0],resize_shape[1],1,), kernel_size=3, activation='relu',name='conv_1'))
convnet.add(Conv2D(filters=16, kernel_size=3, activation='relu',name='conv_2'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_1'))
convnet.add(Conv2D(filters=32, kernel_size=3, activation='relu',name='conv_3'))
convnet.add(Conv2D(filters=64, kernel_size=3, activation='relu',name='conv_4'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_2'))
convnet.add(Conv2D(filters=128, kernel_size=3, activation='relu',name='conv_5'))
convnet.add(Conv2D(filters=256, kernel_size=3, activation='relu',name='conv_6'))
convnet.add(MaxPooling2D(pool_size=2,name='pool_3'))
convnet.add(Flatten())
convnet.add(Dense(units=1024, activation='relu',name='dense_1'))

#encode each of the two inputs into a vector with the convnet
encoded_l = convnet(left_input)
encoded_r = convnet(right_input)
#merge two encoded inputs with the l1 distance between them
L1_distance = lambda x: K.abs(x[0]-x[1])
both = merge([encoded_l,encoded_r], mode = L1_distance, output_shape=lambda x: x[0])
prediction = Dense(1,activation='sigmoid')(both)
siamese_net = Model(input=[left_input,right_input],output=prediction)

optimizer=Adam(0.0001)
siamese_net.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])


In [ ]:
print("Baseline accuracy: ",1 - np.mean(binary_dataset['target']))

Baseline accuracy:  0.6528267397832614


In [ ]:
history = siamese_net.fit_generator(siamese_data_generator, verbose=1, epochs = 100, steps_per_epoch=len(binary_dataset)//BATCH_SIZE)

Epoch 1/100
239/239 [==============================] - 161s 672ms/step - loss: 0.6648 - acc: 0.6351
Epoch 2/100
239/239 [==============================] - 163s 680ms/step - loss: 0.6430 - acc: 0.6487
Epoch 3/100
239/239 [==============================] - 160s 669ms/step - loss: 0.6376 - acc: 0.6520
Epoch 4/100
239/239 [==============================] - 163s 680ms/step - loss: 0.6328 - acc: 0.6534
Epoch 5/100
239/239 [==============================] - 162s 677ms/step - loss: 0.6264 - acc: 0.6603
Epoch 6/100
 41/239 [====>.........................] - ETA: 2:15 - loss: 0.6281 - acc: 0.6456

In [38]:
import time
import os

now = time.strftime('%Y.%m.%d %H:%M:%S')
directory = "weights/" + now + "/"
if not os.path.exists(directory):
    os.makedirs(directory)


siamese_net.save_weights(directory + "siamese_weights")
convnet.save_weights(directory + "convnet_weights")